In [15]:
for p in ("Knet","ArgParse","PlyIO","Plots","JLD")
    Pkg.installed(p) == nothing && Pkg.add(p)
end
using PlyIO
using Knet
using ArgParse
using JLD
using Plots



In [27]:
###EXTRA FUNC

function preprocess(dataset, a)
    r = length(dataset)
    c = length(dataset[1])
    datasetmat = zeros(r, c)
    for i=1:r
        for j=1:c
            datasetmat[i,j] = dataset[i][j]
        end
    end
    minarray = minimum(datasetmat,1)
    maxarray = maximum(datasetmat,1)
    denom = maxarray.-minarray
    denom = denom + 1e-6 * (denom.==0)

    for i=1:r
        for j=1:c
            dataset[i][j] = 2 * a * ((dataset[i][j]-minarray[j]) / denom[j]) - a
        end
    end 

    return dataset
end

##LEAKY RELU
function lrelu(x, alpha=0.2)
    pos = max(0,x)
    neg = min(0,x) * alpha
    return pos + neg
end

###EUC DIST, Nearest Neighbor
function nn(x, A)
    smallestdist = norm(x-Array{Float64,2}(A[1]))
    smallestidx = 1
    for i in 1:length(A)
        vec=Array{Float64,2}(A[i])
        if norm(x-vec)<smallestdist
            smallestdist = norm(x-vec)
            smallestidx = i
        end
    end
    closestmodel = A[smallestidx]
    return smallestidx
end

nn (generic function with 1 method)

In [3]:
#####TRIAL

path = "scapecomp/"
##trial
data0 = load_ply("scapecomp/mesh000.ply")
#atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32}
datax = data0["vertex"]["x"]
datay = data0["vertex"]["y"]
dataz = data0["vertex"]["z"]
data1 = hcat(datax,datay)
data1 = hcat(data1,dataz)


files = readdir(path)
#println(files[1])
dataset=Any[]
for i in files
    data0 = load_ply(path * i)
    datax = data0["vertex"]["x"]
    datay = data0["vertex"]["y"]
    dataz = data0["vertex"]["z"]
    data1 = hcat(datax,datay)
    data1 = hcat(data1,dataz)
    data1 = reshape(data1,37500,1)
    dataset = push!(dataset,data1)
end

#println(size(dataset))
#dataset = minibatch(dataset, 1; xtype=atype)
#println(length(dataset))
#println(dataset)
##end of trial

In [4]:
#####NETWORK

const F = Float64

function encode(ϕ, x)
    x = mat(x)
    x = lrelu.(ϕ[1]*x .+ ϕ[2])
    μ = ϕ[3]*x .+ ϕ[4]
    logσ² = ϕ[5]*x .+ ϕ[6]
    return μ, logσ²
end

function decode(θ, z)
    z = lrelu.(θ[1]*z .+ θ[2])
    return tanh.(θ[3]*z .+ θ[4])
end

function binary_cross_entropy(x, x̂)
    s = @. x * log(x̂ + F(1e-10)) + (1-x) * log(1 - x̂ + F(1e-10))
    return -mean(s)
end

function output(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    #KL =  -sum(@. 1 + logσ² - μ*μ - σ²) / 2
    # Normalise by same number of elements as in reconstruction
    #KL /= M*28*28 ###1

    z = μ .+ randn!(similar(μ)) .* σ
    x̂ = decode(θ, z)
    #BCE = binary_cross_entropy(mat(x), x̂) ####2

    return x̂
end

function loss(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    KL =  -sum(@. 1 + logσ² - μ*μ - σ²) / 2
    # Normalise by same number of elements as in reconstruction
    #KL /= M*28*28 ###1

    z = μ .+ randn!(similar(μ)) .* σ
    x̂ = decode(θ, z)
    BCE = mean(abs2,mat(x)-x̂)
    #BCE = binary_cross_entropy(mat(x), x̂) ####2

    return 500*BCE + KL/500
end

function z_out(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)
    z = μ .+ randn!(similar(μ)) .* σ
    return z
end

function mseloss(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    z = μ .+ randn!(similar(μ)) .* σ
    x̂ = decode(θ, z)
    BCE = mean(abs2,mat(x)-x̂)
    #BCE = binary_cross_entropy(mat(x), x̂) ####2

    return BCE
end

function KLloss(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    KL =  -sum(@. 1 + logσ² - μ*μ - σ²) / 2
    # Normalise by same number of elements as in reconstruction
    #KL /= M*28*28 ###1

    return KL
end

function aveloss(θ, ϕ, data) #####3
    ls = F(0)
    nθ = length(θ)
    #for (x, y) in data
    for x in data
        ls += loss([θ; ϕ], x, nθ)
    end
    return ls / length(data)
end

function train!(θ, ϕ, data, opt; epochs=1)
    w = [θ; ϕ]
    for epoch=1:epochs
        for (x, y) in data
            dw = grad(loss)(w, x, length(θ))
            update!(w, dw, opt)
        end
    end
    return θ, ϕ
end

function weights(nz, nh; atype=Array{F})
    θ = [  # z->x
        xavier(nh, nz),
        zeros(nh),
        xavier(12500*3, nh), #x
        zeros(12500*3)
        ]
    θ = map(a->convert(atype,a), θ)

    ϕ = [ # x->z
        xavier(nh, 12500*3),
        zeros(nh),
        xavier(nz, nh), #μ
        zeros(nz),
        xavier(nz, nh), #σ
        zeros(nz)
        ]
    ϕ = map(a->convert(atype,a), ϕ)

    return θ, ϕ
end




weights (generic function with 1 method)

In [5]:
function main(args="")
    s = ArgParseSettings()
    s.description="Variational Auto Encoder on MNIST dataset."
    s.exc_handler=ArgParse.debug_handler
    @add_arg_table s begin
        ("--seed"; arg_type=Int; default=-1; help="random number seed: use a nonnegative int for repeatable results")
        ("--batchsize"; arg_type=Int; default=100; help="minibatch size")
        ("--epochs"; arg_type=Int; default=10; help="number of epochs for training")
        ("--nh"; arg_type=Int; default=300; help="hidden layer dimension")
        ("--nz"; arg_type=Int; default=128; help="encoding dimention")
        ("--lr"; arg_type=Float64; default=1e-5; help="learning rate")
        ("--atype"; default=(gpu()>=0 ? "KnetArray{F}" : "Array{F}"); help="array type: Array for cpu, KnetArray for gpu")
        ("--infotime"; arg_type=Int; default=2; help="report every infotime epochs")
    end
    isa(args, String) && (args=split(args))
    if in("--help", args) || in("-h", args)
        ArgParse.show_help(s; exit_when_done=false)
        return
    end
    o = parse_args(args, s; as_symbols=true)

    atype = eval(parse(o[:atype]))
    info("using ", atype)
    o[:seed] > 0 && setseed(o[:seed])

    θ, ϕ = weights(o[:nz], o[:nh], atype=atype)
    w = [θ; ϕ]
    opt = optimizers(w, Adam, lr=o[:lr])

    path = "scapecomp/"
    files = readdir(path)
    dataset=Any[]
    for i in files
        data0 = load_ply(path * i)
        datax = data0["vertex"]["x"]
        datay = data0["vertex"]["y"]
        dataz = data0["vertex"]["z"]
        data1 = hcat(datax,datay)
        data1 = hcat(data1,dataz)
        data1=data1/2
        data1[:,2]=data1[:,2]+0.5
        data1[:,3]=data1[:,3]+0.5 
        data1 = reshape(data1,37500,1)
        dataset = push!(dataset,atype(data1))
    end
    
    path2 = "scape_test/"
    files = readdir(path2)
    testdata=Any[]
    for i in files
        data0 = load_ply(path2 * i)
        datax = data0["vertex"]["x"]
        datay = data0["vertex"]["y"]
        dataz = data0["vertex"]["z"]
        data1 = hcat(datax,datay)
        data1 = hcat(data1,dataz)
        data1=data1/2
        data1[:,2]=data1[:,2]+0.5
        data1[:,3]=data1[:,3]+0.5 
        data1 = reshape(data1,37500,1)
        testdata = push!(testdata,atype(data1))
    end
    
    ######dataset = preprocess(dataset, 0.9)
    
    #println(typeof(dataset))
    #println(size(dataset))
    #dataset = reshape(dataset,12500,3,1,72)
    #println(size(dataset))
    #dataset = minibatch(dataset,1;xtype=atype())

    #for (d1,d2) in dataset
    #    println(d1)
    #    break
    #end
    #println(size(dataset[1]))
    randomout = output(w, atype(reshape(dataset[1],37500,1)), length(θ))
    #return reshape(randomout,12500,3)

    #####TRAINING PART#######
    report(epoch) = begin
            #dtrn = minibatch(xtrn, ytrn, o[:batchsize]; xtype=atype)
            #dtst = minibatch(xtst, ytst, o[:batchsize]; xtype=atype)
            println((:epoch, epoch,
                     :trn, aveloss(θ, ϕ, dataset),
                     :length, length(dataset)))
    end

    report(0); tic()
    ls_list = []
    lsTest_list = Any[]
    zlist = Any[]
    
    @time for epoch=1:o[:epochs]
        ls = 0
        kldiv = 0
        for x in dataset      
            ls += mseloss(w, atype(reshape(x,37500,1)), length(θ))
            kldiv += KLloss(w, atype(reshape(x,37500,1)), length(θ))
        end
        
        lsTest = 0
        for x in testdata
            lsTest += mseloss(w, atype(reshape(x,37500,1)), length(θ))
        end 
        
        for x in dataset      
            dw = grad(loss)(w, atype(reshape(x,37500,1)), length(θ))
            update!(w, dw, opt)
        end               
        
        println(kldiv/length(dataset))
        println(ls/length(dataset))
        println(lsTest/length(testdata))
        push!(ls_list, ls/length(dataset))
        push!(lsTest_list, lsTest/length(testdata))
        (epoch % o[:infotime] == 0) && (report(epoch); toc(); tic())
        #println(23)
    end; toq()
    for x in dataset      
            z = z_out(w, atype(reshape(x,37500,1)), length(θ))
            push!(zlist,z)
        end
    
    samplein = dataset[1]
    sampleout = output(w, atype(reshape(dataset[1],37500,1)), length(θ))
    sampleout = reshape(sampleout,12500,3)
    return θ, ϕ, reshape(randomout,12500,3), w, sampleout, samplein, ls_list, lsTest_list, zlist

end

#return randomout

#end # module

p1, p2, randomout, w, sampleout, samplein, ls_list, lsTest_list, zlist = main("--infotime 1 --seed 1 --epochs 1000 --lr 1e-5");

gpu()

INFO: using Knet.KnetArray{Float64,N} where N


(:epoch, 0, :trn, 126.97832631399648, :length, 63)
4.308913407337199
0.25396589948891485
0.26800147080559006
(:epoch, 1, :trn, 42.24565231804756, :length, 63)
elapsed time: 7.17534617 seconds
2650.1843804437235
0.0740589019914723
0.07696360327887777
(:epoch, 2, :trn, 11.561128554757365, :length, 63)
elapsed time: 3.728353442 seconds
2815.044085856403
0.011848611889397318
0.011489066236114997
(:epoch, 3, :trn, 9.24503796622361, :length, 63)
elapsed time: 3.72629892 seconds
2186.7199894185187
0.009740926298974446
0.009129645938273593
(:epoch, 4, :trn, 8.149824509559338, :length, 63)
elapsed time: 3.730741612 seconds
1728.3584526931438
0.009365854307559469
0.008902052748162803
(:epoch, 5, :trn, 7.456568614185245, :length, 63)
elapsed time: 3.725686683 seconds
1430.000128193316
0.009176579287538187
0.008559268486072878
(:epoch, 6, :trn, 6.975854102350022, :length, 63)
elapsed time: 3.734532017 seconds
1218.587388933714
0.009082004527243632
0.008455810698455916
(:epoch, 7, :trn, 6.633665204

(:epoch, 57, :trn, 2.8061844172562203, :length, 63)
elapsed time: 3.716226069 seconds
246.61096156732674
0.004629286805476758
0.003813468188231615
(:epoch, 58, :trn, 2.8428364260883905, :length, 63)
elapsed time: 3.723571933 seconds
259.6559066650298
0.004580974294639162
0.003829266034490045
(:epoch, 59, :trn, 2.7651866374546947, :length, 63)
elapsed time: 3.729488602 seconds
252.17050384739096
0.00449669416038347
0.0037785146606297802
(:epoch, 60, :trn, 2.761117106289951, :length, 63)
elapsed time: 3.726374151 seconds
242.3992632067985
0.004642254754100701
0.0041200358950587036
(:epoch, 61, :trn, 2.7339745385222343, :length, 63)
elapsed time: 3.735283522 seconds
248.16352491469274
0.004465613426189197
0.0036296825157988745
(:epoch, 62, :trn, 2.6965311806938006, :length, 63)
elapsed time: 3.73141826 seconds
240.57853077124827
0.004427701167984932
0.0037702609982350104
(:epoch, 63, :trn, 2.674144845323681, :length, 63)
elapsed time: 3.74744105 seconds
247.9984417238901
0.004387156120361

(:epoch, 120, :trn, 1.952890823857488, :length, 63)
elapsed time: 3.759809682 seconds
194.70790600082265
0.0032145680450785244
0.003055821870397771
(:epoch, 121, :trn, 1.732188739521705, :length, 63)
elapsed time: 3.739837133 seconds
202.920136376779
0.0026808308911483707
0.0026167430857379154
(:epoch, 122, :trn, 1.8619483216927015, :length, 63)
elapsed time: 3.73390604 seconds
197.97853649912267
0.002869189153880622
0.0029174977974606355
(:epoch, 123, :trn, 1.829445628088142, :length, 63)
elapsed time: 3.728259198 seconds
208.36203506684825
0.002807846341389558
0.0026349561076394997
(:epoch, 124, :trn, 1.7274064897578179, :length, 63)
elapsed time: 3.72956226 seconds
203.21844261897036
0.002582404551573421
0.002695944428158003
(:epoch, 125, :trn, 1.788236447118248, :length, 63)
elapsed time: 3.723417281 seconds
206.58879745374566
0.0026970719165277084
0.0027146144294963705
(:epoch, 126, :trn, 1.8053545461653833, :length, 63)
elapsed time: 3.728012444 seconds
211.30936459017312
0.00268

198.19743526504374
0.0018138911176139153
0.0018958760268406386
(:epoch, 176, :trn, 1.2338380321281743, :length, 63)
elapsed time: 3.742240355 seconds
200.3158379781316
0.0016898191774505143
0.0018396951121811434
(:epoch, 177, :trn, 1.3876940292038524, :length, 63)
elapsed time: 3.735188836 seconds
204.70898495217241
0.0019758811609712807
0.002195561560761782
(:epoch, 178, :trn, 1.22419861270728, :length, 63)
elapsed time: 3.718122182 seconds
196.61639456372805
0.0017030367242409873
0.0018282626855492066
(:epoch, 179, :trn, 1.4666489965616218, :length, 63)
elapsed time: 3.725334288 seconds
190.3855816549228
0.0021421541191662336
0.0023407340200099843
(:epoch, 180, :trn, 1.252289156943587, :length, 63)
elapsed time: 3.731936878 seconds
201.2226445503297
0.0017073191029620303
0.0019311897181865391
(:epoch, 181, :trn, 1.495140218190762, :length, 63)
elapsed time: 3.735070697 seconds
188.3311132421542
0.0021689340162396108
0.0020411929790235934
(:epoch, 182, :trn, 1.2451214292697679, :lengt

...

161.2998860014096
0.0004829068295005664
0.0008974202533370537
(:epoch, 782, :trn, 0.5971585078954886, :length, 63)
elapsed time: 3.722267237 seconds
164.71536002075698
0.0004921455782842793
0.0009652891803688284
(:epoch, 783, :trn, 0.5870043226211824, :length, 63)
elapsed time: 3.724012899 seconds
164.85630224046872
0.0005138507239890018
0.0010418743873299833
(:epoch, 784, :trn, 0.5323506451634166, :length, 63)
elapsed time: 3.729260655 seconds
164.14534519162632
0.00045875611772837647
0.0008962597789759145
(:epoch, 785, :trn, 0.697081950446168, :length, 63)
elapsed time: 3.722502896 seconds
168.37455369766707
0.0007595504848086116
0.0011448208934433219
(:epoch, 786, :trn, 0.6058465266253292, :length, 63)
elapsed time: 3.722449872 seconds
166.18984521711218
0.0005615590899670375
0.0009121533139068999
(:epoch, 787, :trn, 0.6064764923486939, :length, 63)
elapsed time: 3.725610138 seconds
163.44829910837618
0.0006197250559257672
0.0009406080200631974
(:epoch, 788, :trn, 0.6329822600182006

161.51959774538273
0.0006290735892696782
0.0009500198432454788
(:epoch, 837, :trn, 0.5780500549474699, :length, 63)
elapsed time: 3.753539151 seconds
162.44886546467555
0.000428550556314502
0.0008545696332214662
(:epoch, 838, :trn, 0.5640959723017834, :length, 63)
elapsed time: 3.750989579 seconds
161.51973927551893
0.00048737653407979664
0.0008875810734411877
(:epoch, 839, :trn, 0.5895467629880131, :length, 63)
elapsed time: 3.769080875 seconds
159.8275307760245
0.0005497895767066209
0.0009484000822810761
(:epoch, 840, :trn, 0.5711279158863272, :length, 63)
elapsed time: 3.760119704 seconds
164.1332040006906
0.0005291679464601443
0.0009623074741452742
(:epoch, 841, :trn, 0.5348934855979287, :length, 63)
elapsed time: 3.763674697 seconds
161.14033105246435
0.00045471869109256496
0.0009592227722231276
(:epoch, 842, :trn, 0.6373931322679335, :length, 63)
elapsed time: 3.76897836 seconds
164.56148964171126
0.0006259027756204742
0.001036538580461782
(:epoch, 843, :trn, 0.5503145277257713, 

162.1862669360127
0.0008468820436903208
0.0012181357070384226
(:epoch, 892, :trn, 0.5852786383403563, :length, 63)
elapsed time: 3.748802349 seconds
158.53371990555726
0.00046939262033708596
0.0009614963035449856
(:epoch, 893, :trn, 0.5507179140811567, :length, 63)
elapsed time: 3.726931234 seconds
156.04640852553382
0.0004652151765325648
0.0010886941032163628
(:epoch, 894, :trn, 0.5841100986218686, :length, 63)
elapsed time: 3.724929214 seconds
159.46329011344372
0.0005145483864093725
0.0009685462878122301
(:epoch, 895, :trn, 0.5688355658982102, :length, 63)
elapsed time: 3.738977911 seconds
160.61139175099117
0.0004910133064051522
0.0008709901446664454
(:epoch, 896, :trn, 0.5132397004958994, :length, 63)
elapsed time: 3.740404502 seconds
159.4132039038918
0.00040557754948755014
0.000741100026372401
(:epoch, 897, :trn, 0.5481134861123167, :length, 63)
elapsed time: 3.734174979 seconds
158.11219690506394
0.00047130730803136085
0.0009856529129064371
(:epoch, 898, :trn, 0.523568379515733

162.94375659637024
0.0007157477033685781
0.0010693248319408575
(:epoch, 947, :trn, 0.5378130637885525, :length, 63)
elapsed time: 3.726194472 seconds
160.86380950918982
0.0004795185707621922
0.0009021342817876149
(:epoch, 948, :trn, 0.5139099778245475, :length, 63)
elapsed time: 3.729965823 seconds
160.07997136860726
0.0004235145648002127
0.0008427772806328131
(:epoch, 949, :trn, 0.5824351767130701, :length, 63)
elapsed time: 3.729874288 seconds
162.4277150229905
0.0005874384411601545
0.0007638083270657753
(:epoch, 950, :trn, 0.5442257499893616, :length, 63)
elapsed time: 3.738890311 seconds
160.51937592018587
0.0004523999696843979
0.0008874113451548027
(:epoch, 951, :trn, 0.5806242185889249, :length, 63)
elapsed time: 3.750616975 seconds
158.5055302824719
0.0005776732548530265
0.0010873362654363523
(:epoch, 952, :trn, 0.6929527569515798, :length, 63)
elapsed time: 3.744997893 seconds
160.46296611647927
0.0006563595937073684
0.0011571836479459035
(:epoch, 953, :trn, 0.5612155336998897,

0

In [6]:
println(size(randomout))
sampleout=Array{Float64,2}(sampleout)
writedlm("test5.xyz", sampleout)
#println(randomout)

samplein=reshape(samplein,12500,3)
samplein=Array{Float64,2}(samplein)
println(maximum(samplein[:,1]), minimum(samplein[:,1]))
println(maximum(samplein[:,2]), minimum(samplein[:,2]))
maximum(samplein[:,3]), minimum(samplein[:,3])
writedlm("samplein.xyz", samplein)
#dataset[1]

(12500, 3)
0.92395001649856570.007438099943101406
0.58156800270080570.41107550263404846


In [20]:
x1 = Array{F}(zlist[1])
y1 = Array{F}(zlist[50])
scatter(x1,y1,reuse=false)

In [8]:
plot(ls_list)
plot!(lsTest_list)


In [9]:
ls_list

1000-element Array{Any,1}:
 0.253966   
 0.0740589  
 0.0118486  
 0.00974093 
 0.00936585 
 0.00917658 
 0.009082   
 0.00904075 
 0.00895281 
 0.00899032 
 0.00913441 
 0.00894839 
 0.00896328 
 ⋮          
 0.000433613
 0.000593278
 0.000497569
 0.000447214
 0.000667179
 0.000407423
 0.000489957
 0.000438174
 0.000424493
 0.000523058
 0.00042246 
 0.000462231

In [10]:
atype = KnetArray{F}
path = "scapecomp/"
    files = readdir(path)
    dataset=Any[]
    for i in files
        data0 = load_ply(path * i)
        datax = data0["vertex"]["x"]
        datay = data0["vertex"]["y"]
        dataz = data0["vertex"]["z"]
        data1 = hcat(datax,datay)
        data1 = hcat(data1,dataz)
        data1=data1/2
        data1[:,2]=data1[:,2]+0.5
        data1[:,3]=data1[:,3]+0.5 
        data1 = reshape(data1,37500,1)
        dataset = push!(dataset,atype(data1))
    end
samplein = dataset[1]
thet, yy = weights(128, 300, atype=atype)
for i in 1:length(dataset)
    sample = dataset[i]
    sampleout = output(w, atype(reshape(dataset[i],37500,1)), length(thet))
    sampleout = reshape(sampleout,12500,3)
    sampleout=Array{Float64,2}(sampleout)
    writedlm("outputDL/meshout"*string(i)*".xyz", sampleout)
    writedlm("outputDL/meshin"*string(i)*".xyz", reshape(Array{Float64,2}(sample),12500,3))
end

In [17]:
zlistFloat = zeros(Float64,length(zlist),length(zlist[1]))
for i in 1:length(zlist)
    for j in 1:length(zlist[1])
        zlistFloat[i,j] = zlist[i][j]
    end
end
zlistFloat[:,1]
x1 = Array{F}(zlistFloat[:,5])
y1 = Array{F}(zlistFloat[:,128])
scatter(x1,y1,reuse=false)



In [31]:
nθ = length(thet)
θ, ϕ = w[1:nθ], w[nθ+1:end]
z2out = decode(θ, zlist[37])
z2out = reshape(z2out,12500,3)
z2out=Array{Float64,2}(z2out)
writedlm("z2out1.xyz", z2out)
z2out = decode(θ, zlist[13])
z2out = reshape(z2out,12500,3)
z2out=Array{Float64,2}(z2out)
writedlm("z2out2.xyz", z2out)
z2out = decode(θ, (zlist[1]+zlist[28])/2)
z2out = reshape(z2out,12500,3)
z2out=Array{Float64,2}(z2out)
writedlm("nearest2.xyz", z2out)

In [13]:
#p1, p2, randomout, w, sampleout, samplein, ls_list, lsTest_list, zlist

save("model_kl500.jld", "w", w, "zlist", zlist, "ls_list", ls_list, "lsTest_list", lsTest_list)


In [30]:
nn(reshape(z2out,37500,1), dataset)
nearestdata1 = reshape(dataset[13],12500,3)
nearestdata1 = Array{Float64,2}(nearestdata1)
writedlm("nearestdata1.xyz", nearestdata1)


In [39]:
#Pkg.add("Meshes")
using Meshes
#mesh_0149.obj
m = load("mesh_0149.obj")
save("my3dmodel_now_a.ply", m)
data0 = load_ply("my3dmodel_now_a.ply")
datax = data0["vertex"]["x"]
datay = data0["vertex"]["y"]
dataz = data0["vertex"]["z"]
data1 = hcat(datax,datay)
data1 = hcat(data1,dataz)

10002×3 Array{Float32,2}:
 0.215958    0.045607    0.290787  
 0.072543    0.0525721  -0.53662   
 0.0734126   0.0495714  -0.510908  
 0.0862569   0.0444856  -0.504181  
 0.203248    0.0305674   0.256582  
 0.220985    0.0486874   0.298073  
 0.405693    0.040475    0.189623  
 0.442535    0.0397477   0.151818  
 0.323796    0.0433699   0.281435  
 0.436168    0.0258002   0.118725  
 0.345016    0.0265142  -0.480993  
 0.346823    0.0309439  -0.49144   
 0.292614    0.0442542  -0.527232  
 ⋮                                 
 0.273888    0.901688    0.0576709 
 0.222007    0.944051    0.0419116 
 0.261149    0.983943    0.00882474
 0.232166    0.978189    0.00627649
 0.00756438  1.19568    -0.271808  
 0.281657    0.992542    0.00235238
 0.312896    1.01053    -0.00858099
 0.297023    0.996404   -0.00922817
 0.273182    0.987783    0.0122897 
 0.297479    1.00086    -0.0015035 
 0.3113      0.998047   -0.014889  
 0.305895    1.0128      0.00136752